# Data van alle NS-stations in NL

## samenvatting

In dit notebook beschrijven we hoe we met behuilp van de NS-API alle stations, coordinaten en de bestemming en vertrektijd van de volgende vertrekkende trein. Deze data zetten we vervolgens in een MySQL datatbase voor later gebruik.

## Uitwerking

### Stations ophalen

Om de NS-api te gebruiken moet er eerst een account woden aangemaakt op het NS-API portal https://apiportal.ns.nl/. HIerna kunnen we met het NS voorbeeldscript via de /reisinformatie-api/api/v2/stations een json object met alle stations  ophalen.

nb. vul je eigen API-key in om de code te testen!

In [1]:
from __future__ import print_function
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

#Vul hier je eigen ns API-key in
api_key = ''

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': api_key
}

params = urllib.parse.urlencode({
    # Request parameters
    'lang': 'nl',
    'countries': 'nl',
    'name': 'stations',
    'land': 'NL'})
try:
    conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
    conn.request("GET", "/reisinformatie-api/api/v2/stations?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    conn.close()
except Exception as e:
    print(("[Errno {0}] {1}".format(e.errno, e.strerror)))
    
#de voglende stap moeten we doen om de nested dictionary met key 'payload' uit de json data te halen en op te slaan als variable stations.
stations = json.loads(data)['payload']

### Definieer functie om de bestemming en vertrektijd van de volgende trein op te halen van de API

Onderstaand definieren we twee functies (`get_next_departure_time(station_dep)` en `get_next_departure_direction(station_dep)`). Hiermee kunnen we de benodigde data ophalen uit de  /reisinformatie-api/api/v2/departures api. De functies hebben de stationscode nodig als input en geven de vertrektijd en eindbestamming als resultaat.


In [2]:
#define funtion to retrieve next departure time:
def get_next_departure_time(station_dep):
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'uicCode': station_dep,
        'maxJourneys': 1})
    try:
        conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
        conn.request("GET", "/reisinformatie-api/api/v2/departures?%s" % params, "{body}", headers)
        response = conn.getresponse()
        #Extract the departure time from the response:
        departure_time = json.loads(response.read())['payload']['departures'][0]['plannedDateTime']
        conn.close()

    except Exception:
        #In case no data is returned by the API, return NULL from the function
        return('NULL') 
       
    return(departure_time)


####################################################

#define funtion to retrieve next departure destination:
def get_next_departure_direction(station_dep):
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': '2280b63fa9cd4919b60175ced7235a74'
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'uicCode': station_dep,
        'maxJourneys': 1})
    try:
        conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
        conn.request("GET", "/reisinformatie-api/api/v2/departures?%s" % params, "{body}", headers)
        response = conn.getresponse()
        #Extract the destination from the response:
        departure_direction = json.loads(response.read())['payload']['departures'][0]['direction']
        conn.close()

    except Exception:
        #In case no data is returned by the API, return NULL from the function
        return('NULL')
    
    return(departure_direction)



Nu de functies zijn gedefinieerd om de data te verrijken kunnen we verbinding maken met de database, de data verrijken en opslaan in de DB!

### Data verrijken met bestemming en vertrektijd van de volgende trein en opslaan in de DB

in onderstaand blok maken we de verbinding met de databse-server en maken we de tabel `tbl_stations` aan. Op de server moet van te voren een lege database ns_data zijn aangemaakt.   
Vervolgens halen we mbv. van de twee hiervoor gecreëerde functies de vertrektijd en bestamming van de volgende trein op dat station (in NL) op, voegen dat toe aan de stations dictionary en zetten vervolgens de gewenste data uit de stations dict in de database.


nb. om de code te testen moet je de inlog en adres van je eigen SQL server invullen en een lege db nd_data aangemaakt hebben.


In [3]:
import mysql.connector
from mysql.connector import errorcode

try:
#connect to mysql db with the provided credentials (Change to your own when evaluating the code!)
    cnx = mysql.connector.connect(user='root',
                                password='',
                                host='localhost',
                                database='ns_data')      #create empty db before running this script!
except mysql.connector.Error as err:                           #Return error message in case credentials are invalid
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
        exit()
    elif err.errno == errorcode.ER_BAD_DB_ERROR:               #return error message in case database doesn't exist
        print("Database does not exist")
        exit()
    else:
        print(err)                                             #print error message in case of error message not one of the two above
else:
    cursor = cnx.cursor()


#create the required table    (You need to create an empty database 'ns_data' before running this script!) 
cursor.execute("CREATE TABLE `tbl_stations` (`stations_code` int NOT NULL, `naam_kort` varchar(50) NOT NULL, `naam_middel` varchar(50) NOT NULL, `naam_lang` varchar(50) NOT NULL,  `latitude` float NOT NULL, `longitude` float NOT NULL, `volgende_bestemming` varchar(50), `volgend_vertrek` varchar(30)) ENGINE=InnoDB")
cnx.commit()

##########

for station in stations:
    if station['land'] == 'NL':
        #haal volgende eindbestemming en vertrektijd op en voeg deze aan de json data toe van de stations:
        station['volgende_eindbestemming'] = get_next_departure_direction(station['UICCode'])
        station['volgende_verterktijd'] = get_next_departure_time(station['UICCode'])
        #Zet in database (in nieuwe nieuwe for loop)
        cursor.execute("INSERT INTO `tbl_stations` (stations_code, naam_kort, naam_middel, naam_lang, latitude, longitude, volgende_bestemming, volgend_vertrek) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", ((int(station['UICCode'])), station['namen']['kort'], station['namen']['middel'], station['namen']['lang'], station['lat'], station['lng'], station['volgende_eindbestemming'], station['volgende_verterktijd']))
        cnx.commit()
            
# close sonnection and prina message.
cursor.close()
cnx.close()
print("De data is succesvol geïmporteerd in de MySQL database!")

De data is succesvol geïmporteerd in de MySQL database!
